In [1]:
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import requests
import shutil
import urllib
import PIL.Image as Image
import pandas as pd
from IPython.display import display
import numpy as np
from os import listdir
from os.path import isfile, join

from sklearn.model_selection import train_test_split
from pathlib import Path
from utils.utils import plot_results


In [2]:
data = {}
mypath = "train/"

files_path=[join(mypath, k) for k in listdir(mypath)]
print(files_path)
anna_s=sorted(listdir(files_path[1]))[:200]
img_s=sorted(listdir(files_path[0]))[:200]


for ann_path, img_path in zip(anna_s,img_s):
    data[ann_path]=img_path    
        
data

['train/image', 'train/annos']


{'000001.json': '000001.jpg',
 '000002.json': '000002.jpg',
 '000003.json': '000003.jpg',
 '000004.json': '000004.jpg',
 '000005.json': '000005.jpg',
 '000006.json': '000006.jpg',
 '000007.json': '000007.jpg',
 '000008.json': '000008.jpg',
 '000009.json': '000009.jpg',
 '000010.json': '000010.jpg',
 '000011.json': '000011.jpg',
 '000012.json': '000012.jpg',
 '000013.json': '000013.jpg',
 '000014.json': '000014.jpg',
 '000015.json': '000015.jpg',
 '000016.json': '000016.jpg',
 '000017.json': '000017.jpg',
 '000018.json': '000018.jpg',
 '000019.json': '000019.jpg',
 '000020.json': '000020.jpg',
 '000021.json': '000021.jpg',
 '000022.json': '000022.jpg',
 '000023.json': '000023.jpg',
 '000024.json': '000024.jpg',
 '000025.json': '000025.jpg',
 '000026.json': '000026.jpg',
 '000027.json': '000027.jpg',
 '000028.json': '000028.jpg',
 '000029.json': '000029.jpg',
 '000030.json': '000030.jpg',
 '000031.json': '000031.jpg',
 '000032.json': '000032.jpg',
 '000033.json': '000033.jpg',
 '000034.j

In [3]:
clothing=[]
for ann in data:
#     print(type(ann))
    ann_path = files_path[1]+"/"+ann
    img_path = files_path[0]+"/"+data[ann]
    k=json.load(open(ann_path))
    clothing.append([k,img_path])

clothing[0]

[{'item2': {'segmentation': [[460,
     438,
     374,
     484,
     251,
     520,
     269,
     586,
     298,
     622,
     410,
     623,
     410,
     567,
     413,
     591,
     420,
     623,
     465,
     622,
     456,
     561,
     466,
     504,
     460,
     438],
    [374, 484, 251, 520, 269, 586, 298, 622, 410, 623, 410, 567, 374, 484],
    [460,
     438,
     374,
     484,
     410,
     567,
     413,
     591,
     420,
     623,
     465,
     622,
     456,
     561,
     466,
     504,
     460,
     438]],
   'scale': 2,
   'viewpoint': 2,
   'zoom_in': 3,
   'landmarks': [251,
    520,
    1,
    374,
    484,
    1,
    460,
    438,
    1,
    269,
    586,
    1,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    410,
    567,
    1,
    413,
    591,
    2,
    0,
    0,
    0,
    0,
    0,
    0,
    456,
    561,
    2,
    0,
    0,
    0],
   'style': 0,
   'bounding_box': [249, 423, 466, 623],
   'category_

In [4]:
cat = ["short sleeve top","long sleeve top","short sleeve outwear","long sleeve outwear","vest","sling","shorts","trousers","skirt", "short sleeve dress", "long sleeve dress", "vest dress","sling dress"]
cat

['short sleeve top',
 'long sleeve top',
 'short sleeve outwear',
 'long sleeve outwear',
 'vest',
 'sling',
 'shorts',
 'trousers',
 'skirt',
 'short sleeve dress',
 'long sleeve dress',
 'vest dress',
 'sling dress']

In [5]:
train_clothing, val_clothing = train_test_split(clothing, test_size=0.1)
len(train_clothing), len(val_clothing)

(180, 20)

In [6]:
def create_dataset(clothing, categories, dataset_type):

    images_path = Path(f"dress/images/{dataset_type}")
    labels_path = Path(f"dress/labels/{dataset_type}")
    
    for img_id, rtest in enumerate(tqdm(clothing)):
        image_name = f"{img_id}.jpeg"
#         img = cv2.imread(rtest[1])
        img = Image.open(rtest[1])
        img = img.convert('RGB')
        img.save(str(images_path / image_name), "JPEG")
        
        img_width, img_height = img.size

        label_name = f"{img_id}.txt"
        with (labels_path / label_name).open(mode="w") as label_file:
            l = len(rtest[0])-2 # removing source and pair id for item count 
            # rtest[0]["item1"]
            for i in range(1,l+1):
                pts = rtest[0][f"item{i}"]["bounding_box"]
                top_l = pts[:2]
                lower_r = pts[2:]
                
                top_l_n = [pts[0]/img_width, pts[1]/img_height]
                lower_r_n = [pts[2]/img_width, pts[3]/img_height]
                
                w = lower_r_n[0]-top_l_n[0]
                h = lower_r_n[1]-top_l_n[1]
#                 tl = [top_l[0]/]
#                 print("pts     :",pts)
#                 print("top_l   :",top_l)
#                 print("lower_r :",lower_r)
                label = rtest[0][f"item{i}"]["category_name"]
                category_idx = categories.index(label)
                label_file.write(
                f"{category_idx} {top_l_n[0] + w / 2} {top_l_n[1] + h / 2} {w} {h}\n")



In [7]:
create_dataset(train_clothing, cat, 'train')
create_dataset(val_clothing, cat, 'val')

100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 29.06it/s]


In [8]:
!tree dress -L 2

dress
├── images
│   ├── train
│   └── val
└── labels
    ├── train
    └── val

6 directories, 0 files


In [9]:
# !git clone https://github.com/ultralytics/yolov5
%cd yolov5

/Users/rohitreddypanyam/myntra/dev/yolov5


In [10]:
# !!!!!!!! normalize the values.
!python train.py --img 640 --batch 4 --epochs 30 \
  --data data/dress.yaml --cfg models/yolov5x.yaml --weights yolov5x.pt \
  --name yolov5x_dress --cache

train: weights=yolov5x.pt, cfg=models/yolov5x.yaml, data=data/dress.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=30, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=yolov5x_clothing, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 1 commit. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-52-g8a803f3 torch 1.8.0 CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv

In [ ]:
plot_results()

In [ ]:
!find ../dress/images/val/ -maxdepth 1 -type f | head -50 | xargs cp -t "./inference/images/"

In [ ]:
# predecting usig model

!python detect.py --weights weights/best_yolov5x_dress.pt \
  --img 640 --conf 0.4 --source ./inference/images/